In [1]:
#웹드라이버 매니저 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
ChromeDriverManager().install()

'/Users/mac/.wdm/drivers/chromedriver/mac64/126.0.6478.182/chromedriver-mac-arm64/chromedriver'

In [2]:
from selenium import webdriver
browser = webdriver.Chrome()

url= 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)


    


In [3]:

# 1페이지의 링크 데이터 전부 수집


#    https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24
link_list = []
for i in range(1,4):
    print(f'현재 {i}페이지 수집중 입니다.')
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    
    ## 한개의 베스트 셀러 링크 수집
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    ### 1페이지 전체의 링크데이터
    #browser.find_element(By.CLASS_NAME,'gd_name') #엘리먼트 : 요소
    datas = browser.find_elements(By.CLASS_NAME,'gd_name') #엘리먼트들: 리스트

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
            
    time.sleep(2)

print(link_list)

현재 1페이지 수집중 입니다.
현재 2페이지 수집중 입니다.
현재 3페이지 수집중 입니다.
['https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/104663596', 'https://www.yes24.com/Product/Goods/128133897', 'https://www.yes24.com/Product/Goods/124225606', 'https://www.yes24.com/Product/Goods/123930880', 'https://www.yes24.com/Product/Goods/127094902', 'https://www.yes24.com/Product/Goods/123930891', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/126511412', 'https://www.yes24.com/Product/Goods/124043802', 'https://www.yes24.com/Product/Goods/126845471', 'https://www.yes24.com/Product/Goods/125295101', 'https://www.yes24.com/Product/Goods/126702038', 'https://www.yes24.com/Product/Goods/126516935', 'https://www.yes24.com/Product/Goods/126590468', 'https://www.yes24.com/Product/Goods/8759796', 'https://www.yes24.

In [4]:
len(link_list)

72

In [5]:
# 상세페이지로 이동
#for link in link_list:
#    browser.get(link)

browser.get(link_list[0])
title = browser.find_element(By.CLASS_NAME,'gd_name').text
author = browser.find_element(By.CLASS_NAME,'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME,'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME,'gd_date').text
rating = browser.find_element(By.CLASS_NAME,'yes_b').text
reviews = browser.find_element(By.CLASS_NAME,'txC_blue').text
sales = browser.find_element(By.CLASS_NAME,'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME,'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[0]
ranking_weeks = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[1]


In [6]:
#상세페이지 이동 후 데이터 크롤링
import pymysql      #pip3 install pymysql
import time
import re
from datetime import datetime

title = browser.find_element(By.CLASS_NAME,'gd_name').text
author = browser.find_element(By.CLASS_NAME,'gd_auth').text


publisher = browser.find_element(By.CLASS_NAME,'gd_pub').text
# 2020년 12월 25일 -> 2023-12-25 로 변경해줘야함
publishing = browser.find_element(By.CLASS_NAME,'gd_date').text
match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
if match:
    year, month, day = match.groups()
    data_obj = datetime(int(year),int(month),int(day))
    publishing = data_obj.strftime("%Y-%m-%d")
else:
    publishing = "9999-01-01"

rating = browser.find_element(By.CLASS_NAME,'yes_b').text
reviews = browser.find_element(By.CLASS_NAME,'txC_blue').text
reviews = int(reviews.replace(',',''))

sales = browser.find_element(By.CLASS_NAME,'gd_sellNum').text.split(" ")[2]
sales = int(sales.replace(',',''))

price = browser.find_element(By.CLASS_NAME,'yes_m').text[:-1]
price = int(price.replace(',',''))


ranking = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[0].split(" ")[2].split('위')[0]
ranking_weeks = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[1].split(" ")[3][0]

print(title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks)

너에게 들려주는 단단한 말 김종원 저 퍼스트펭귄 2024-07-10 10.0 7 205758 18000 1 1


In [7]:

#데이터베이스 연동후 -> 수집한 데이터를 디비에 저장 (CSV)
import pymysql      #pip3 install pymysql
import time
import re
from datetime import datetime

conn = pymysql.connect (
    host = 'localhost',
    user = 'root',
    password= '0000',
    db= 'yes24',
    charset = 'utf8mb4',
    cursorclass = pymysql.cursors.DictCursor
    )


link_list = []
for i in range(1,4):
    print(f'현재 {i}페이지 수집중 입니다.')
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    
    ## 한개의 베스트 셀러 링크 수집
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    ### 1페이지 전체의 링크데이터
    #browser.find_element(By.CLASS_NAME,'gd_name') #엘리먼트 : 요소
    datas = browser.find_elements(By.CLASS_NAME,'gd_name') #엘리먼트들: 리스트

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
    time.sleep(2)

    
with conn.cursor() as cur:
    
    for link in link_list:
        try:
            browser.get(link)
            
            title = browser.find_element(By.CLASS_NAME,'gd_name').text
            author = browser.find_element(By.CLASS_NAME,'gd_auth').text

            publisher = browser.find_element(By.CLASS_NAME,'gd_pub').text
            # 2020년 12월 25일 -> 2023-12-25 로 변경해줘야함
            publishing = browser.find_element(By.CLASS_NAME,'gd_date').text
            match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
            if match:
                year, month, day = match.groups()
                data_obj = datetime(int(year),int(month),int(day))
                publishing = data_obj.strftime("%Y-%m-%d")
            else:
                publishing = "9999-01-01"

            rating = browser.find_element(By.CLASS_NAME,'yes_b').text
            reviews = browser.find_element(By.CLASS_NAME,'txC_blue').text
            reviews = int(reviews.replace(',',''))

            sales = browser.find_element(By.CLASS_NAME,'gd_sellNum').text.split(" ")[2]
            sales = int(sales.replace(',',''))

            price = browser.find_element(By.CLASS_NAME,'yes_m').text[:-1]
            price = int(price.replace(',',''))


            ranking = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[0].split(" ")[2].split('위')[0]
            ranking_weeks = browser.find_element(By.CLASS_NAME,'gd_best').text.split('|')[1].split(" ")[3][0]
            print(title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks)
            
            sql = """INSERT INTO BOOKS(
                title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks
                )
                VALUES(
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """
            cur.execute(sql,(title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks))
            conn.commit()
            time.sleep(2)
        except Exception as e:
            print(e) 








현재 1페이지 수집중 입니다.
현재 2페이지 수집중 입니다.
현재 3페이지 수집중 입니다.
너에게 들려주는 단단한 말 김종원 저 퍼스트펭귄 2024-07-10 10.0 7 205758 18000 1 1
하루 한 장 나의 어휘력을 위한 필사 노트 유선경 저 위즈덤하우스 2024-03-28 9.6 198 563304 23800 2 1
불변의 법칙 모건 하우절 저/이수경 역 서삼독 2024-02-28 9.5 365 590346 25000 3 6
언젠가 우리가 같은 별을 바라본다면 차인표 저 해결책 2021-12-15 9.8 92 98706 12000 4 1
죽이고 싶은 아이 2 이꽃님 저 우리학교 2024-07-01 9.9 42 170934 14000 5 3
내 몸 혁명 박용우 저 루미너스 2024-01-10 9.8 68 222297 19500 6 3
2024 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상 최태성 저 이투스북 2023-12-01 9.7 76 408654 16000 7 1
그의 운명에 대한 아주 개인적인 생각 유시민 저 생각의길 2024-06-19 9.1 377 764904 16800 8 4
2024 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 하 최태성 저 이투스북 2023-12-01 10.0 64 386181 15500 9 1
세이노의 가르침 세이노(SayNo) 저 데이원 2023-03-02 9.0 2344 1156308 7200 10 2
ETS 토익 정기시험 기출문제집 1000 Vol. 4 RC ETS 저 YBM(와이비엠) 2023-12-20 9.6 156 444642 19800 11 2
신화의 숲 김헌 저 포레스트북스 2024-05-20 9.8 62 174858 18000 12 6
ETS 토익 정기시험 기출문제집 1000 Vol. 4 LC ETS 저 YBM(와이비엠) 2023-12-20 9.8 137 415053 19800 13 2
허송세월 김훈 저 나남 2024-06-20 9.3 45 221046 18